# Bitcoin price forecasting with PySpark - Data crawling
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



# Dependencies, Libraries and Tools

In [2]:
import pandas as pd
import functools

from google.colab import drive

#  Define metrics and parameters

In this section we are going to define the parameters used to collect the data and the metrics used. I will consider Bitcoin data for 10 years, starting from 2012-01-01 through 2022-12-31.

Note that since the Blockchain.com API allows retreiving data with a maximum timespan equal to 6 years, I manually computed the continue date so that I could make a second API call to get the remaining data.

Regarding the metrics, I chose the ones that seemed most relevant to me, containing both price statistics but also technical features of Bitcoin's blockchain.

In [7]:
from datetime import date

# Get the current date
current_date = date.today()

# Print the current date
print(current_date)

2023-08-18


In [8]:
# Define the parameters
timespan = "6years" # Duration of the data (because the Max timespan == 6years)
start_date = "2016-01-01"
continue_date = "2021-12-31" # The continue date (manually calculate the continue_date)
end_date = str(current_date)

# Metrics considered
metrics = [
          ##Currency Statistics##
            "market-price", # Market Price: The average USD market price across major bitcoin exchanges.
            "trade-volume", #Exchange Trade Volume (USD): The total USD value of trading volume on major bitcoin exchanges.

          ##Block Details##
            "blocks-size", # Blockchain Size (MB): The total size of the blockchain minus database indexes in megabytes.
            "avg-block-size", # Average Block Size (MB): The average block size over the past 24 hours in megabytes.
            "n-transactions-total", # Total Number of Transactions: The total number of transactions on the blockchain.
            "n-transactions-per-block", # Average Transactions Per Block: The average number of transactions per block over the past 24 hours.

          ##Mining Information##
            "hash-rate", # Total Hash Rate (TH/s): The estimated number of terahashes per second the bitcoin network is performing in the last 24 hours.
            "difficulty", # Network Difficulty (T): A relative measure of how difficult it is to mine a new block for the blockchain.
            "miners-revenue", # Miners Revenue (USD): Total value of coinbase block rewards and transaction fees paid to miners.
            "transaction-fees-usd", # Total Transaction Fees (USD): The total USD value of all transaction fees paid to miners. This does not include coinbase block rewards.

          ##Network Activity##
            "n-unique-addresses", # The total number of unique addresses used on the blockchain.
            "n-transactions", # Confirmed Transactions Per Day: The total number of confirmed transactions per day.
            "estimated-transaction-volume-usd" # Estimated Transaction Value (USD): The total estimated value in USD of transactions on the blockchain. This does not include coins returned as change.
]

# Retreiving data

In this section we are going to make the call to the Blockchain.com API to retrieve the data.

In [9]:
def data_crawler(timespan, metrics, start_date, continue_date, end_date):
    # API Info
    url1 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={start_date}&format=csv'
    url2 = f'https://api.blockchain.info/charts/{metrics}?timespan={timespan}&start={continue_date}&format=csv'

    # Obtain Data
    data1 = pd.read_csv(url1,names=['timestamp',metrics])
    data2 = pd.read_csv(url2,names=['timestamp',metrics])

    # Concat by rows
    all_data = pd.concat([data1,data2])

    # Transform "timestamp" to datetime type
    all_data['timestamp'] = pd.to_datetime(all_data["timestamp"])

    # Keep the same end date with Bitcoin data
    all_data = all_data[(all_data['timestamp'] < end_date)]

    return all_data

In [10]:
# Merge the data
merge = functools.partial(pd.merge, on='timestamp')

# Gain Blockchain Data from Blockchain.com API
df1 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df1

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07
2,2016-01-03,432.76,1.287046e+06,54754.876205,0.554656,101427625,983.503448,7.694240e+05,1.038803e+11,1.720316e+06,11009.108568,335666.0,142608.0,6.764693e+07
3,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07
4,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2774,2023-08-13,29412.84,2.358204e+07,503041.771369,1.700897,878051199,3682.311111,3.515913e+08,5.239118e+13,2.558566e+07,401312.894785,650366.0,497112.0,1.306994e+09
2775,2023-08-14,29283.84,2.631797e+07,503271.735112,1.712082,878550205,3324.660819,4.453489e+08,5.239118e+13,3.233787e+07,539304.002744,723900.0,568517.0,3.301793e+09
2776,2023-08-15,29413.80,8.428877e+07,503564.249680,1.655429,879116599,3409.725146,4.453489e+08,5.239118e+13,3.191514e+07,536032.415741,786113.0,583063.0,2.891347e+09
2777,2023-08-16,29169.09,7.327130e+07,503847.295317,1.685333,879700157,3477.779141,4.245139e+08,5.239118e+13,3.058697e+07,530443.731065,669749.0,566878.0,2.674221e+09


In [11]:
# Check duplicated rows
len(df1['timestamp'].unique())

2779

Due to a problem with the Blockchain.com API, I was forced to make an additional call to retrieve capitalization and total circulating data that will be added to the currency statistics to get a single dataset.

In [12]:
# Retrieving market capitalization and total circulating data
metrics = [
  "total-bitcoins", # Total Circulating Bitcoin: The total number of mined bitcoin that are currently circulating on the network.
  "market-cap", # Market Capitalization (USD): The total USD value of bitcoin in circulation.
  ]

merge = functools.partial(pd.merge, on='timestamp')
df2 = functools.reduce(merge, [data_crawler(timespan, metric, start_date, continue_date, end_date) for metric in metrics])
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01 00:03:21,15029575.00,6.474140e+09
1,2016-01-02 15:54:15,15035125.00,6.499685e+09
2,2016-01-04 02:26:37,15040650.00,6.458253e+09
3,2016-01-05 09:50:06,15046150.00,6.489359e+09
4,2016-01-06 21:03:44,15051750.00,6.472554e+09
...,...,...,...
3000,2023-08-16 09:53:22,19458950.00,5.672284e+11
3001,2023-08-16 18:29:58,19459312.50,5.670054e+11
3002,2023-08-17 02:54:20,19459675.00,5.565078e+11
3003,2023-08-17 11:16:17,19460037.50,5.549419e+11


In [13]:
# Check duplicated rows
len(df2['timestamp'].unique())

3005

In [14]:
# Wipe off the timestamp's h:m:s.
df2['timestamp'] = pd.to_datetime(df2["timestamp"]).dt.normalize()

# Drop the duplicates in column "timestamp", keep the last value
df2.drop_duplicates(subset="timestamp", keep="last", inplace=True)

In [15]:
df2

,timestamp,total-bitcoins,market-cap
0,2016-01-01,15029575.00,6.474140e+09
1,2016-01-02,15035125.00,6.499685e+09
2,2016-01-04,15040650.00,6.458253e+09
3,2016-01-05,15046150.00,6.489359e+09
4,2016-01-06,15051750.00,6.472554e+09
...,...,...,...
2993,2023-08-13,19456400.00,5.702671e+11
2995,2023-08-14,19457131.25,5.768650e+11
2998,2023-08-15,19458218.75,5.718381e+11
3001,2023-08-16,19459312.50,5.670054e+11


In [16]:
# Check duplicated rows
len(df2['timestamp'].unique())

2099

In [17]:
# Add the market capitalization and total circulating data
all_data = pd.merge(df1, df2, how="inner", on='timestamp')
all_data = all_data.interpolate(method='ffill')
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
3,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
4,2016-01-06,432.43,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08,15051750.00,6.472554e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,2023-08-13,29412.84,2.358204e+07,503041.771369,1.700897,878051199,3682.311111,3.515913e+08,5.239118e+13,2.558566e+07,401312.894785,650366.0,497112.0,1.306994e+09,19456400.00,5.702671e+11
2091,2023-08-14,29283.84,2.631797e+07,503271.735112,1.712082,878550205,3324.660819,4.453489e+08,5.239118e+13,3.233787e+07,539304.002744,723900.0,568517.0,3.301793e+09,19457131.25,5.768650e+11
2092,2023-08-15,29413.80,8.428877e+07,503564.249680,1.655429,879116599,3409.725146,4.453489e+08,5.239118e+13,3.191514e+07,536032.415741,786113.0,583063.0,2.891347e+09,19458218.75,5.718381e+11
2093,2023-08-16,29169.09,7.327130e+07,503847.295317,1.685333,879700157,3477.779141,4.245139e+08,5.239118e+13,3.058697e+07,530443.731065,669749.0,566878.0,2.674221e+09,19459312.50,5.670054e+11


In [18]:
# Check nan value
all_data[all_data.isnull().T.any()]

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap


In [19]:
# Check duplicated rows
len(all_data['timestamp'].unique())

2095

In [20]:
all_data

,timestamp,market-price,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd,total-bitcoins,market-cap
0,2016-01-01,430.89,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07,15029575.00,6.474140e+09
1,2016-01-02,434.75,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07,15035125.00,6.499685e+09
2,2016-01-04,430.78,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07,15040650.00,6.458253e+09
3,2016-01-05,434.17,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08,15046150.00,6.489359e+09
4,2016-01-06,432.43,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08,15051750.00,6.472554e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,2023-08-13,29412.84,2.358204e+07,503041.771369,1.700897,878051199,3682.311111,3.515913e+08,5.239118e+13,2.558566e+07,401312.894785,650366.0,497112.0,1.306994e+09,19456400.00,5.702671e+11
2091,2023-08-14,29283.84,2.631797e+07,503271.735112,1.712082,878550205,3324.660819,4.453489e+08,5.239118e+13,3.233787e+07,539304.002744,723900.0,568517.0,3.301793e+09,19457131.25,5.768650e+11
2092,2023-08-15,29413.80,8.428877e+07,503564.249680,1.655429,879116599,3409.725146,4.453489e+08,5.239118e+13,3.191514e+07,536032.415741,786113.0,583063.0,2.891347e+09,19458218.75,5.718381e+11
2093,2023-08-16,29169.09,7.327130e+07,503847.295317,1.685333,879700157,3477.779141,4.245139e+08,5.239118e+13,3.058697e+07,530443.731065,669749.0,566878.0,2.674221e+09,19459312.50,5.670054e+11


In [21]:
new_columns = ['timestamp', 'market-price', 'total-bitcoins', 'market-cap'] + [col for col in all_data.columns if col not in ['timestamp', 'market-price', 'total-bitcoins', 'market-cap']]
all_data = all_data.reindex(columns=new_columns)
all_data

,timestamp,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
0,2016-01-01,430.89,15029575.00,6.474140e+09,2.860153e+06,54604.791735,0.493407,101155706,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.0,124092.0,5.834626e+07
1,2016-01-02,434.75,15035125.00,6.499685e+09,1.646042e+06,54670.707780,0.579744,101278339,1027.848276,7.074570e+05,1.038803e+11,1.671420e+06,13798.467773,333102.0,149038.0,5.083235e+07
2,2016-01-04,430.78,15040650.00,6.458253e+09,1.967359e+06,54835.983701,0.556970,101571729,1001.955801,9.036860e+05,1.038803e+11,2.076921e+06,13811.521698,344268.0,181354.0,9.627657e+07
3,2016-01-05,434.17,15046150.00,6.489359e+09,2.484225e+06,54936.400034,0.641779,101752002,1161.598726,8.417189e+05,1.038803e+11,1.819808e+06,14331.031533,359763.0,182371.0,1.031559e+08
4,2016-01-06,432.43,15051750.00,6.472554e+09,1.677504e+06,55036.950659,0.662380,101934175,1220.496454,7.022931e+05,1.038803e+11,1.702648e+06,14113.278937,312004.0,172090.0,1.007001e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,2023-08-13,29412.84,19456400.00,5.702671e+11,2.358204e+07,503041.771369,1.700897,878051199,3682.311111,3.515913e+08,5.239118e+13,2.558566e+07,401312.894785,650366.0,497112.0,1.306994e+09
2091,2023-08-14,29283.84,19457131.25,5.768650e+11,2.631797e+07,503271.735112,1.712082,878550205,3324.660819,4.453489e+08,5.239118e+13,3.233787e+07,539304.002744,723900.0,568517.0,3.301793e+09
2092,2023-08-15,29413.80,19458218.75,5.718381e+11,8.428877e+07,503564.249680,1.655429,879116599,3409.725146,4.453489e+08,5.239118e+13,3.191514e+07,536032.415741,786113.0,583063.0,2.891347e+09
2093,2023-08-16,29169.09,19459312.50,5.670054e+11,7.327130e+07,503847.295317,1.685333,879700157,3477.779141,4.245139e+08,5.239118e+13,3.058697e+07,530443.731065,669749.0,566878.0,2.674221e+09


Once we have the daily dataset we will go to sample it at a frequency of 1 minute (1T) using the resample method. This means that the data will be organized in 1-minute time-frame, and an interpolation method will be used to fill in any missing data or holes in the DataFrame by estimating missing values based on the surrounding known values.

In [22]:
# Upsampling to 1min by interpolate
all_data.set_index('timestamp', inplace=True)
all_data_30m = all_data.resample('30T').interpolate()
all_data_30m

,market-price,total-bitcoins,market-cap,trade-volume,blocks-size,avg-block-size,n-transactions-total,n-transactions-per-block,hash-rate,difficulty,miners-revenue,transaction-fees-usd,n-unique-addresses,n-transactions,estimated-transaction-volume-usd
timestamp,,,,,,,,,,,,,,,
2016-01-01 00:00:00,430.890000,1.502958e+07,6.474140e+09,2.860153e+06,54604.791735,0.493407,1.011557e+08,919.200000,6.971292e+05,1.038803e+11,1.554769e+06,8783.063851,263121.000000,124092.000000,5.834626e+07
2016-01-01 00:30:00,430.970417,1.502969e+07,6.474672e+09,2.834859e+06,54606.164986,0.495206,1.011583e+08,921.463506,6.973443e+05,1.038803e+11,1.557199e+06,8887.551433,264578.937500,124611.708333,5.818972e+07
2016-01-01 01:00:00,431.050833,1.502981e+07,6.475204e+09,2.809565e+06,54607.538237,0.497004,1.011608e+08,923.727011,6.975595e+05,1.038803e+11,1.559629e+06,8992.039015,266036.875000,125131.416667,5.803318e+07
2016-01-01 01:30:00,431.131250,1.502992e+07,6.475736e+09,2.784271e+06,54608.911488,0.498803,1.011634e+08,925.990517,6.977747e+05,1.038803e+11,1.562059e+06,9096.526597,267494.812500,125651.125000,5.787664e+07
2016-01-01 02:00:00,431.211667,1.503004e+07,6.476268e+09,2.758977e+06,54610.284739,0.500602,1.011659e+08,928.254023,6.979898e+05,1.038803e+11,1.564490e+06,9201.014178,268952.750000,126170.833333,5.772010e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-16 22:00:00,28740.447500,1.946032e+07,5.447720e+11,1.005588e+08,504099.540082,1.647714,8.802218e+08,3241.211296,4.078024e+08,5.239118e+13,2.860487e+07,538270.665912,760807.916667,507657.666667,4.336937e+09
2023-08-16 22:30:00,28730.705625,1.946034e+07,5.442667e+11,1.011790e+08,504105.272917,1.646859,8.802336e+08,3235.834754,4.074226e+08,5.239118e+13,2.855983e+07,538448.550795,762877.437500,506311.750000,4.374726e+09
2023-08-16 23:00:00,28720.963750,1.946036e+07,5.437613e+11,1.017991e+08,504111.005753,1.646004,8.802455e+08,3230.458212,4.070428e+08,5.239118e+13,2.851478e+07,538626.435678,764946.958333,504965.833333,4.412515e+09


# Output

In this last section we are going to save the dataset we just created to the Google Drive.

In [23]:
GDRIVE_DIR = "/content/drive"
GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"

In [24]:
# Link Colab to our Google Drive
drive.mount(GDRIVE_DIR)

Mounted at /content/drive


In [25]:
GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_30min"
GDRIVE_DATASET_NAME_EXT = "/" + GDRIVE_DATASET_NAME + ".parquet"
GDRIVE_DATASET = GDRIVE_DATASET_RAW_DIR + GDRIVE_DATASET_NAME_EXT

In [26]:
# Output the 1h data
all_data_30m.to_parquet(GDRIVE_DATASET)